In [1]:
import pandas as pd
from pulp import *

### Read the given nutrition dataset into a Pandas DataFrame object
Note we are reading all the rows with `nrows=12` argument because we just want to read all the nutrients informtion and not the maximum/minimum bounds in the dataset. We will enter those bounds in the optimization problem separately.

In [2]:
df = pd.read_excel("Data/diet - medium.xls",nrows=12)

### Show first 5 rows of the dataset

In [3]:
df.head()

,Foods,Price/Serving,Serving Size,Calories,Cholesterol (mg),Total_Fat (g),Carbohydrates (g),Dietary_Fiber (g),Protein (g)
0,Chapathi,4.00,1 chapathi,70.0,0.0,0.4,15.0,0.4,3.0
1,White Rice,5.46,1 cup,241.8,0.0,0.4,45.0,0.6,4.3
2,Peanuts,14.60,1 cup,828.0,0.0,72.0,24.0,12.0,38.0
3,Boiled Potatoes,1.00,1 Potato,87.0,0.0,0.1,20.1,1.8,1.9
4,Ground Nut,20.00,1 cup,729.0,0.0,12.0,121.0,35.0,39.0


### Create the `PuLP` problem variable. Since it is a cost minimization problem, we need to use `LpMinimize`

In [4]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Simple Diet Problem",LpMinimize)

C:\Users\Lenovo\Anaconda3\lib\site-packages\pulp\pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### Create a list of food items from the dataset

In [5]:
# Creates a list of the Ingredients
food_items = list(df['Foods'])

In [6]:
print("So, the food items to consdier, are\n"+"-"*100)
for f in food_items:
    print(f,end=', ')

So, the food items to consdier, are
----------------------------------------------------------------------------------------------------
Chapathi, White Rice, Peanuts, Boiled Potatoes, Ground Nut, Cottage Cheese, Roasted Chicken, Raw Apple, Banana, White Bread, Milk, Boiled Eggs, 

### Create a dictinary of costs for all food items

In [7]:
costs = dict(zip(food_items,df['Price/Serving']))

In [8]:
costs

{'Chapathi': 4.0,
 'White Rice': 5.46,
 'Peanuts': 14.6,
 'Boiled Potatoes': 1.0,
 'Ground Nut': 20.0,
 'Cottage Cheese': 18.0,
 'Roasted Chicken': 100.0,
 'Raw Apple': 5.0,
 'Banana': 5.0,
 'White Bread': 1.2,
 'Milk': 10.0,
 'Boiled Eggs': 5.0}

### Create a dictionary of calories for all food items

In [9]:
calories = dict(zip(food_items,df['Calories']))

### Create a dictionary of cholesterol for all food items

In [10]:
cholesterol = dict(zip(food_items,df['Cholesterol (mg)']))

### Create a dictionary of total fat for all food items

In [11]:
fat = dict(zip(food_items,df['Total_Fat (g)']))

### Create a dictionary of carbohydrates for all food items

In [12]:
carbs = dict(zip(food_items,df['Carbohydrates (g)']))

### Create a dictionary of dietary fiber for all food items

In [13]:
fiber = dict(zip(food_items,df['Dietary_Fiber (g)']))

### Create a dictionary of protein for all food items

In [14]:
protein = dict(zip(food_items,df['Protein (g)']))

### Create a dictionary of food portion with lower bound 0 - these are the main optimization variables

In [15]:
# A dictionary called 'food_vars' is created to contain the referenced Variables
food_vars = LpVariable.dicts("Portion",food_items,0,cat='Continuous')

### Create another set of variables for each food, with integer 0 or 1. These are indicator variables

In [16]:
# A dictionary called 'food_vars' is created to contain the referenced Variables
food_chosen = LpVariable.dicts("Chosen",food_items,0,1,cat='Integer')

### Adding the objective function to the problem

In [17]:
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

### Adding the calorie constraints to the problem

In [18]:
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 3600.0, "CalorieMinimum"
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 4000.0, "CalorieMaximum"

### Adding other nutrient constraints to the problem one by one...

In [19]:
# Fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 40.0, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 70.0, "FatMaximum"

# Carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 328.0, "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 574.0, "CarbsMaximum"

# Fiber
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) >= 60.0, "FiberMinimum"
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) <= 125.0, "FiberMaximum"

# Protein
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= 180.0, "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= 200.0, "ProteinMaximum"

### Adding constraint linking `food_vars` and `food_chosen`

In [20]:
for f in food_items:
    prob += food_vars[f]>= food_chosen[f]*0.1
    prob += food_vars[f]<= food_chosen[f]*1e5

### Adding constraint of celery and frozen broccoli

In [21]:
prob += food_vars['White Bread']<=8

In [22]:
prob += food_vars['Cottage Cheese']<=1

In [23]:
prob += food_vars['Milk']<=2

In [24]:
prob += food_vars['Boiled Potatoes']<=8

In [25]:
prob += food_vars['Boiled Eggs']<=4

In [26]:
prob += food_vars['Ground Nut']<=2

### Adding constraint of at least 3 types of protein in every diet

In [27]:
protein_choices = ['White Rice','Peanuts','Boiled Eggs','Roasted Chicken',
                  'Milk']

In [28]:
prob += lpSum([food_chosen[p] for p in protein_choices]) >= 3.0

### Writing problem data to a `.lp` file

In [29]:
# The problem data is written to an .lp file
prob.writeLP("SimpleDietProblem.lp")

[Chosen_Banana,
 Chosen_Boiled_Eggs,
 Chosen_Boiled_Potatoes,
 Chosen_Chapathi,
 Chosen_Cottage_Cheese,
 Chosen_Ground_Nut,
 Chosen_Milk,
 Chosen_Peanuts,
 Chosen_Raw_Apple,
 Chosen_Roasted_Chicken,
 Chosen_White_Bread,
 Chosen_White_Rice,
 Portion_Banana,
 Portion_Boiled_Eggs,
 Portion_Boiled_Potatoes,
 Portion_Chapathi,
 Portion_Cottage_Cheese,
 Portion_Ground_Nut,
 Portion_Milk,
 Portion_Peanuts,
 Portion_Raw_Apple,
 Portion_Roasted_Chicken,
 Portion_White_Bread,
 Portion_White_Rice]

### Run the solver

In [30]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

### Print the problem solution status `'optimal'`, `'infeasible'`, `'unbounded'` etc...

In [31]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


### Scan through the problem variables and print out only if the variable quanity is positive i.e. it is included in the optimal solution

In [32]:
print("The optimal (least cost) balanced diet with additional constraints \
(e.g. at least 3 types of animal protein sources, consists of\n"+"-"*110)
for v in prob.variables():
    if v.varValue>0 and v.name[0]=='P':
        print(v.name, "=", v.varValue)

The optimal (least cost) balanced diet with additional constraints (e.g. at least 3 types of animal protein sources, consists of
--------------------------------------------------------------------------------------------------------------
Portion_Boiled_Eggs = 3.41412
Portion_Boiled_Potatoes = 2.77025
Portion_Chapathi = 7.47362
Portion_Cottage_Cheese = 1.0
Portion_Ground_Nut = 2.0
Portion_Milk = 2.0
Portion_Peanuts = 0.1
Portion_White_Bread = 8.0
Portion_White_Rice = 0.846703


### Print the optimal diet cost

In [33]:
print("The total cost of this balanced diet is: Rs{}".format(round(value(prob.objective),2)))

The total cost of this balanced diet is: Rs143.42
